# EVALUATION

In [ ]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import transforms, models
from torchvision.datasets.folder import default_loader
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import matplotlib.pyplot as plt


class CustomImageDataset(Dataset):
    def __init__(self, image_dir, label_json, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.labels = json.load(open(label_json))
        self.image_paths = []

       
        for root, _, files in os.walk(image_dir):
            for file in files:
                if file.lower().endswith((".png", ".jpg", ".jpeg")):
                    rel_path = os.path.relpath(os.path.join(root, file), image_dir)
                    self.image_paths.append(rel_path)

        self.class_names = sorted(list(set(self.labels.values())))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.class_names)}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_rel_path = self.image_paths[idx]
        img_path = os.path.join(self.image_dir, img_rel_path)

        
        class_folder = img_rel_path.split(os.sep)[0]  

        
        label_name = self.labels[class_folder]

        label = self.class_to_idx[label_name]

        image = default_loader(img_path)
        if self.transform:
            image = self.transform(image)

        return image, label



class LinearEvaluationModel(nn.Module):
    def __init__(self, simclr_ckpt_path, num_classes):
        super(LinearEvaluationModel, self).__init__()
        
        base_model = models.resnet18(pretrained=False)
        base_model.fc = nn.Identity()  
        self.encoder = base_model

        
        ckpt = torch.load(simclr_ckpt_path, map_location='cpu')
        state_dict = ckpt['model_state_dict']
        new_state_dict = {k.replace("encoder.", ""): v for k, v in state_dict.items() if "encoder" in k}
        self.encoder.load_state_dict(new_state_dict, strict=False)

        
        for param in self.encoder.parameters():
            param.requires_grad = False

      
        self.linear = nn.Linear(512, num_classes)

    def forward(self, x):
        with torch.no_grad():
            features = self.encoder(x)
        return self.linear(features)


transform = transforms.Compose([
    transforms.Resize(128),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(96),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = CustomImageDataset(
    image_dir="/kaggle/input/ssl-dataset/ssl_dataset/val.X",
    label_json="/kaggle/input/ssl-dataset/ssl_dataset/Labels.json",
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LinearEvaluationModel(
    simclr_ckpt_path="/kaggle/input/checkpt_100_model/pytorch/default/1/checkpoint_epoch_100.pt",
    num_classes=100
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.linear.parameters(), lr=1e-3)
epochs = 50
scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
train_losses     = []
learning_rates   = []
epoch_accuracies = []
epoch_f1_scores  = []

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        x, y = x.to(device), y.to(device)
        preds = model(x)
        loss = criterion(preds, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)

    scheduler.step()
    lr = scheduler.get_last_lr()[0]
    learning_rates.append(lr)
    model.eval()
    all_p, all_t = [], []
    with torch.no_grad():
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb).argmax(1).cpu().tolist()
            all_p += out
            all_t += yb.cpu().tolist()

    acc = accuracy_score(all_t, all_p)
    f1  = f1_score(all_t, all_p, average='weighted')
    epoch_accuracies.append(acc)
    epoch_f1_scores.append(f1)

    print(f"Epoch {epoch+1:>2}: Loss={avg_loss:.4f} | LR={lr:.6f} | Acc={acc:.4f} | F1={f1:.4f}")

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for x, y in tqdm(train_loader, desc="Final Eval"):
        x, y = x.to(device), y.to(device)
        outs = model(x).argmax(1).cpu().tolist()
        all_preds.extend(outs)
        all_labels.extend(y.cpu().tolist())

final_acc = accuracy_score(all_labels, all_preds)
final_f1  = f1_score(all_labels, all_preds, average='weighted')

# -------Plotting -------
epochs_range = range(1, epochs+1)
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(epochs_range, train_losses,   marker='o', label='Train Loss')
plt.plot(epochs_range, learning_rates, marker='x', label='Learning Rate')
plt.xlabel('Epoch'); plt.legend(loc='upper right')
plt.title('Loss & LR Schedule')

plt.subplot(1,2,2)
plt.plot(epochs_range, epoch_accuracies, marker='s', label='Accuracy')
plt.plot(epochs_range, epoch_f1_scores,  marker='d', label='F1 Score')
plt.xlabel('Epoch'); plt.legend(loc='lower right')
plt.title('Per‐Epoch Metrics')

plt.tight_layout()
plt.show()
